Tri Utami Gita Cahyani / 4112322012 / Statistika Terapan dan Komputasi

# Import Library

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import joblib

# Load Dataset

## Load Dataset Training

In [2]:
df_train = pd.read_csv("data_training.csv")
df_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.3,0.740,0.08,1.7,0.094,10.0,45.0,0.99576,3.24,0.50,9.8,5,1366
1,8.1,0.575,0.22,2.1,0.077,12.0,65.0,0.99670,3.29,0.51,9.2,5,103
2,10.1,0.430,0.40,2.6,0.092,13.0,52.0,0.99834,3.22,0.64,10.0,7,942
3,12.9,0.500,0.55,2.8,0.072,7.0,24.0,1.00012,3.09,0.68,10.9,6,811
4,8.4,0.360,0.32,2.2,0.081,32.0,79.0,0.99640,3.30,0.72,11.0,6,918


Pada bagian ini, dataset pelatihan (data_training.csv) dimuat menggunakan fungsi pd.read_csv() dari pustaka pandas.

## Info Data Training

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857 entries, 0 to 856
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         857 non-null    float64
 1   volatile acidity      857 non-null    float64
 2   citric acid           857 non-null    float64
 3   residual sugar        857 non-null    float64
 4   chlorides             857 non-null    float64
 5   free sulfur dioxide   857 non-null    float64
 6   total sulfur dioxide  857 non-null    float64
 7   density               857 non-null    float64
 8   pH                    857 non-null    float64
 9   sulphates             857 non-null    float64
 10  alcohol               857 non-null    float64
 11  quality               857 non-null    int64  
 12  Id                    857 non-null    int64  
dtypes: float64(11), int64(2)
memory usage: 87.2 KB


Berikut adalah penjelasan setiap bagian dari output tersebut:
1. RangeIndex: 857 entries, 0 to 856: Dataset memiliki 857 baris data yang terindeks mulai dari 0 hingga 856.
2. Data columns (total 13 columns): Dataset memiliki 13 kolom, yang terdiri dari 11 kolom dengan tipe data numerik (float64) dan 2 kolom dengan tipe data integer (int64).
3. Non-Null Count: Setiap kolom menunjukkan jumlah nilai yang tidak kosong (non-null). Di sini, setiap kolom memiliki 857 non-null, yang menunjukkan bahwa tidak ada nilai hilang (missing value) dalam dataset untuk setiap kolom.
4. Dtype: Ini menunjukkan tipe data untuk setiap kolom. Sebagian besar kolom memiliki tipe data float64, yang digunakan untuk angka desimal, sedangkan dua kolom terakhir (quality dan Id) memiliki tipe data int64, yang digunakan untuk angka bulat.

In [4]:
df_train["quality"].unique()

array([5, 7, 6, 4, 8, 3])

Nilai yang dihasilkan dari perintah df_train["quality"].unique() menunjukkan bahwa variabel quality dalam data pelatihan memiliki enam kelas unik, yaitu 3, 4, 5, 6, 7, dan 8.

In [5]:
print(df_train.quality.value_counts())

quality
5    362
6    341
7    109
4     26
8     13
3      6
Name: count, dtype: int64


Output dari df_train.quality.value_counts() menunjukkan distribusi jumlah sampel untuk setiap kelas pada variabel target quality. Nilai kualitas yang paling banyak muncul adalah 5 sebanyak 362 sampel, diikuti oleh nilai 6 sebanyak 341 sampel. Kemudian disusul oleh nilai 7 sebanyak 109 sampel, nilai 4 sebanyak 26 sampel, nilai 8 sebanyak 13 sampel, dan nilai 3 hanya sebanyak 6 sampel. Distribusi ini menggambarkan bahwa data tidak seimbang (imbalanced), karena terdapat perbedaan jumlah yang cukup signifikan antara kelas dengan frekuensi tertinggi dan terendah. Ketidakseimbangan seperti ini dapat memengaruhi performa model klasifikasi, khususnya terhadap kelas-kelas minoritas, yang cenderung lebih sulit diprediksi dengan akurat.

## Statistika Deskriptif

In [6]:
df_train.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
count,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000
mean,8.261960,0.529393,0.267351,2.506184,0.086830,15.782964,45.978413,0.996692,3.313092,0.656709,10.430338,5.653442,813.749125
std,1.701992,0.179162,0.195144,1.293512,0.048721,10.300402,31.692113,0.001901,0.152079,0.167364,1.066971,0.821777,463.807063
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.390000,8.400000,3.000000,0.000000
25%,7.100000,0.395000,0.090000,1.900000,0.070000,7.000000,21.000000,0.995520,3.210000,0.550000,9.500000,5.000000,413.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,14.000000,38.000000,0.996680,3.310000,0.620000,10.200000,6.000000,814.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,63.000000,0.997800,3.400000,0.730000,11.100000,6.000000,1214.000000
max,15.600000,1.580000,1.000000,15.500000,0.611000,68.000000,278.000000,1.003200,4.010000,2.000000,14.000000,8.000000,1597.000000


Fungsi df_train.describe() memberikan statistik deskriptif untuk setiap kolom numerik dalam dataset, seperti nilai rata-rata, deviasi standar, nilai minimum, maksimum, dan kuartil.

# Data Preparation

## Cek Missing Value

In [7]:
df_train.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


Pada langkah Cek Missing Value, fungsi df_train.isnull().sum() digunakan untuk memeriksa apakah ada nilai yang hilang (missing value) dalam dataset. Fungsi ini akan mengembalikan jumlah nilai yang hilang (NaN) untuk setiap kolom dalam dataset. Jika hasilnya menunjukkan bahwa jumlah nilai yang hilang untuk setiap kolom adalah 0, maka dapat disimpulkan bahwa tidak terjadi missing value pada dataset tersebut. Dengan demikian, tidak perlu melakukan imputasi atau penghapusan data yang hilang, dan proses analisis atau pemodelan dapat dilanjutkan tanpa penanganan missing value.

In [8]:
df_train = df_train.drop(columns=['Id'])
df_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.3,0.740,0.08,1.7,0.094,10.0,45.0,0.99576,3.24,0.50,9.8,5
1,8.1,0.575,0.22,2.1,0.077,12.0,65.0,0.99670,3.29,0.51,9.2,5
2,10.1,0.430,0.40,2.6,0.092,13.0,52.0,0.99834,3.22,0.64,10.0,7
3,12.9,0.500,0.55,2.8,0.072,7.0,24.0,1.00012,3.09,0.68,10.9,6
4,8.4,0.360,0.32,2.2,0.081,32.0,79.0,0.99640,3.30,0.72,11.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
852,6.7,1.040,0.08,2.3,0.067,19.0,32.0,0.99648,3.52,0.57,11.0,4
853,8.0,0.390,0.30,1.9,0.074,32.0,84.0,0.99717,3.39,0.61,9.0,5
854,7.4,0.350,0.33,2.4,0.068,9.0,26.0,0.99470,3.36,0.60,11.9,6
855,7.9,0.570,0.31,2.0,0.079,10.0,79.0,0.99677,3.29,0.69,9.5,6


## Feature Scalling

In [9]:
scaler = MinMaxScaler()

min_max_df = scaler.fit_transform(df_train.drop("quality", axis = 1))
min_max_df = pd.DataFrame(min_max_df, columns = df_train.columns[:-1])

Perintah di atas melakukan normalisasi atau penskalaan pada fitur data df_train menggunakan MinMaxScaler. Fungsi fit_transform mengubah setiap fitur dalam df_train sehingga nilainya berada dalam rentang [0, 1].

## Fitur dan Target

In [10]:
X = min_max_df
y = df_train['quality']

oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

X.shape, y.shape

y.value_counts()

,count
quality,
5,362
7,362
6,362
4,362
8,362
3,362


Setelah data fitur X (yang sudah dinormalisasi dengan MinMaxScaler) dan target y (kolom "quality") ditentukan, dilakukan penyeimbangan kelas menggunakan teknik SMOTE (Synthetic Minority Over-sampling Technique). SMOTE menghasilkan data sintetis untuk kelas yang jumlahnya sedikit, sehingga semua kelas memiliki jumlah sampel yang sama, yaitu 362 sampel per kelas. Hasil akhirnya adalah dataset yang seimbang untuk setiap kelas quality

## Split Data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

telah seimbang (X dan y) menjadi dua bagian: data latih (training set) dan data uji (testing set). Sebesar 80% data digunakan untuk melatih model (X_train, y_train), dan 20% sisanya digunakan untuk menguji performa model (X_test, y_test)

# Modelling

## Random Forest

In [12]:
models = {'Random Forest': RandomForestClassifier(random_state=4)
}

evaluation_results = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Hitung metrik evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Simpan hasil dalam daftar
    evaluation_results.append({
        'Model': model_name,
        'Accuracy': round(accuracy, 4),
        'Precision': round(precision, 4),
        'Recall': round(recall, 4),
        'F1 Score': round(f1, 4)
    })
pd.DataFrame(evaluation_results)

,Model,Accuracy,Precision,Recall,F1 Score
0,Random Forest,0.8506,0.8449,0.8506,0.8455


Model Random Forest yang digunakan menunjukkan performa yang cukup baik dengan nilai akurasi sebesar 85,98%.

In [13]:
classifier = RandomForestClassifier(random_state=4)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=4)

Merupakan proses pelatihan (training) model Random Forest Classifier menggunakan data latih (X_train sebagai fitur dan y_train sebagai label).

In [14]:
y_pred=classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.to_numpy().reshape(len(y_test),1)),1))
final_classifier = classifier

[[7 7]
 [6 5]
 [4 4]
 [7 6]
 [3 3]
 [7 7]
 [7 7]
 [8 8]
 [8 8]
 [6 6]
 [7 7]
 [3 3]
 [7 7]
 [8 8]
 [6 5]
 [7 7]
 [8 8]
 [3 3]
 [3 3]
 [5 5]
 [3 3]
 [3 3]
 [7 7]
 [5 5]
 [8 8]
 [4 4]
 [7 7]
 [5 6]
 [3 3]
 [5 5]
 [6 5]
 [7 7]
 [3 3]
 [5 5]
 [7 7]
 [8 8]
 [3 3]
 [3 3]
 [7 7]
 [6 6]
 [7 7]
 [4 4]
 [4 4]
 [8 8]
 [3 3]
 [6 5]
 [4 4]
 [3 3]
 [8 8]
 [5 5]
 [7 7]
 [4 4]
 [7 7]
 [7 6]
 [3 3]
 [5 5]
 [5 5]
 [3 3]
 [3 3]
 [4 4]
 [6 6]
 [4 4]
 [5 6]
 [5 5]
 [3 5]
 [6 6]
 [5 5]
 [8 8]
 [6 5]
 [8 8]
 [4 6]
 [8 8]
 [4 4]
 [4 4]
 [5 5]
 [3 3]
 [3 3]
 [6 6]
 [7 7]
 [8 8]
 [8 8]
 [7 7]
 [7 7]
 [8 8]
 [5 5]
 [8 8]
 [7 6]
 [6 6]
 [3 3]
 [8 8]
 [5 6]
 [8 8]
 [6 7]
 [5 6]
 [8 8]
 [8 8]
 [5 6]
 [3 3]
 [8 8]
 [4 4]
 [7 7]
 [7 7]
 [3 3]
 [5 6]
 [4 4]
 [8 8]
 [8 8]
 [5 5]
 [7 7]
 [8 8]
 [3 3]
 [6 6]
 [6 6]
 [8 8]
 [5 5]
 [6 6]
 [8 8]
 [4 4]
 [4 4]
 [4 4]
 [3 3]
 [6 6]
 [8 8]
 [3 3]
 [3 3]
 [3 3]
 [8 8]
 [6 6]
 [3 3]
 [4 4]
 [4 4]
 [6 5]
 [3 3]
 [7 7]
 [6 5]
 [7 6]
 [7 7]
 [3 3]
 [3 3]
 [4 4]
 [4 6]
 [7 7]
 [5 5]

In [15]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[76  0  0  0  0  0]
 [ 0 68  2  0  0  0]
 [ 3  0 48 11  1  0]
 [ 1  4 18 39 12  0]
 [ 0  0  2  7 76  3]
 [ 0  0  0  0  1 63]]


0.8505747126436781

confusion_matrix menunjukkan jumlah prediksi benar dan salah dari model untuk tiap kelas secara lebih rinci. Sementara itu, accuracy_score menghitung proporsi prediksi yang benar terhadap total data uji, sehingga memberikan gambaran umum seberapa baik model mengenali kelas dengan tepat.

In [16]:
fin_test_data = pd.read_csv('/content/data_testing.csv')
test_data = fin_test_data.drop('Id',axis=1)
test_data = sc.fit_transform(test_data)

In [17]:
final_pred = final_classifier.predict(test_data)
final_pred

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([5, 5, 5, 7, 6, 7, 5, 5, 5, 5, 7, 7, 5, 6, 6, 5, 6, 5, 6, 7, 6, 6,
       5, 7, 5, 5, 7, 7, 5, 5, 5, 5, 7, 5, 5, 5, 5, 6, 7, 6, 7, 6, 7, 6,
       4, 4, 6, 6, 5, 5, 7, 6, 5, 6, 6, 6, 5, 6, 7, 4, 6, 6, 5, 7, 6, 5,
       6, 6, 5, 7, 7, 5, 6, 5, 6, 4, 6, 7, 5, 7, 5, 5, 5, 4, 6, 7, 5, 5,
       5, 5, 4, 5, 5, 5, 5, 5, 5, 7, 6, 5, 5, 6, 5, 6, 5, 5, 7, 7, 5, 7,
       6, 5, 6, 6, 5, 5, 4, 6, 6, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 7, 5, 5,
       7, 5, 8, 5, 5, 5, 5, 5, 7, 5, 6, 5, 7, 5, 5, 5, 5, 5, 7, 6, 7, 5,
       7, 5, 5, 7, 6, 4, 5, 5, 5, 5, 6, 5, 7, 6, 4, 5, 6, 5, 6, 5, 5, 5,
       7, 6, 5, 5, 7, 7, 6, 5, 5, 5, 6, 5, 7, 6, 5, 6, 6, 5, 5, 7, 4, 6,
       7, 7, 5, 6, 5, 5, 5, 5, 6, 5, 7, 7, 5, 7, 5, 5, 6, 5, 7, 5, 6, 6,
       5, 5, 5, 5, 5, 7, 5, 5, 5, 5, 7, 7, 4, 5, 5, 7, 5, 5, 6, 5, 7, 7,
       7, 5, 5, 6, 6, 5, 7, 5, 5, 5, 5, 5, 5, 6, 6, 6, 5, 5, 5, 7, 7, 5,
       5, 5, 5, 7, 5, 7, 5, 7, 6, 6, 5, 6, 5, 5, 4, 5, 5, 7, 5, 5, 5, 7])

In [18]:
hasil_prediksi = pd.read_csv('/content/hasilprediksi_3digitNIMterakhir.csv')
hasil_prediksi

,Id;Quality
0,222;
1,1514;
2,417;
3,754;
4,516;
...,...
281,1147;
282,296;
283,170;
284,1439;


# Simpan File

In [19]:
output = pd.DataFrame({'Id':fin_test_data.Id,'Quality':final_pred})
output.to_csv('hasilprediksi_012.csv', index=False)
filename = "hasilprediksi_012.csv"
print("Your submission was successfully saved!")

Your submission was successfully saved!
